In [1]:
import datasets
import seaborn as sns
sns.set_style("whitegrid")
import matplotlib.pyplot as plt
import numpy as np
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from utils import remove_extra_brackets, CLASSIFICATION_PROMPT
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForMaskedLM
from torch.nn import Linear, BCEWithLogitsLoss
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import Muon

In [2]:
def remove_extra_brackets(input: str) -> str:
    text = input[2:-2]
    text = text.strip()
    return text

In [3]:
# Load multiple CSV files
df = datasets.load_dataset('csv', data_files={
    'train': '../data/train.csv',
    'test': '../data/test.csv'
})

In [4]:
def fix_dataset(row):
    cleaned_prompt = remove_extra_brackets(row['prompt'])
    cleaned_response_a = remove_extra_brackets(row['response_a'])
    cleaned_response_b = remove_extra_brackets(row['response_b'])
    prompt = CLASSIFICATION_PROMPT.format(
        prompt=cleaned_prompt,
        response_a=cleaned_response_a,
        response_b=cleaned_response_b
    )
    return {
        "final_prompt": prompt,
    }

In [5]:
df = df.map(fix_dataset, batched=False).remove_columns(['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b'])

In [6]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilroberta-base")
# model_classification = AutoModelForSequenceClassification.from_pretrained("Fchandar-lab/NeoBERT", num_labels=3)
# model_maskedLM = AutoModelForMaskedLM.from_pretrained("FacebookAI/roberta-base")

In [43]:
class PreferenceDataset(Dataset):
    
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer
        
    def __len__(self):
        return self.dataset.shape[0]
    
    def __getitem__(self, idx):
        row = self.dataset[idx]
        tokenized_prompt = self.tokenizer(row["final_prompt"], padding="max_length", max_length=self.tokenizer.model_max_length, return_tensors='pt',truncation=True)["input_ids"]
        target = torch.tensor([row["winner_model_a"], row["winner_model_b"], row["winner_tie"]])
        
        return tokenized_prompt, target
        

In [44]:
train_dataset = PreferenceDataset(df["train"], tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [48]:
next(iter(train_dataloader))

[tensor([[[    0, 50118,  1185,  ..., 37457,   257,     2]],
 
         [[    0, 50118,  1185,  ...,    83,  1411,     2]],
 
         [[    0, 50118,  1185,  ...,     1,     1,     1]],
 
         [[    0, 50118,  1185,  ...,   306,     4,     2]]]),
 tensor([[0, 1, 0],
         [1, 0, 0],
         [0, 1, 0],
         [0, 1, 0]])]